<a href="https://colab.research.google.com/github/mgino11/Visual_Analytics/blob/main/HW4_DT_608.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment # 4

This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is here, and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [54]:
import pandas as pd
import numpy as np

In [3]:
url = 'https://data.cityofnewyork.us/resource/uvpi-gqnh.json'
trees = pd.read_json(url)
trees.head(5)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39.0,165.0,3025654.0,3.010850e+09


Build a dash app for an arborist studying the health of various tree species ( sps_common) accross each borough

Questions to be answered by the arborist
What proportion of trees are in good , fair or poor health according to the 

1.   What proportion of trees are in good , fair or poor health according to the health variable
2.   Are stewerdas having impact on the health of trees? ( use steward)



In [74]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash.exceptions import  PreventUpdate


In [75]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools
import plotly.express as px

In [76]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [110]:
soql_health = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
         '$select=health,boroname,spc_common,count(tree_id) as count' +\
         '&$group=health,boroname,spc_common').replace(' ', '%20')

In [112]:
soql_health = pd.read_json(soql_health)

In [113]:
soql_health = soql_health.dropna()
print(soql_health)

    health   boroname             spc_common  count
0     Fair      Bronx         American beech      7
1     Fair      Bronx           American elm    248
2     Fair      Bronx   American hophornbeam     28
3     Fair      Bronx      American hornbeam     61
4     Fair      Bronx         American larch      3
..     ...        ...                    ...    ...
995   Good  Manhattan           tulip-poplar     17
996   Good  Manhattan       Turkish hazelnut      7
997   Good  Manhattan  two-winged silverbell      5
998   Good  Manhattan          Virginia pine      2
999   Good  Manhattan         weeping willow      8

[999 rows x 4 columns]


In [81]:
soql_health_bx = soql_health.query('boroname == "Bronx"')
soql_health_br = soql_health.query('boroname == "Brooklyn"')
soql_health_qu = soql_health.query('boroname == "Queens"')
soql_health_mn = soql_health.query('boroname == "Manhattan"')
soql_health_si = soql_health.query('boroname == "Staten Island"')

In [116]:
soql_health_steward = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
         '$select=health,boroname,steward,spc_common,count(tree_id) as count' +\
        '&$group=health,boroname,steward,spc_common').replace(' ', '%20')

In [117]:
soql_health_steward = pd.read_json(soql_health_steward)

In [118]:
soql_health_steward = soql_health_steward.dropna()
#print(soql_health_steward)

In [85]:
soql_health_steward_bx = soql_health_steward.query('boroname == "Bronx"')
soql_health_steward_br = soql_health_steward.query('boroname == "Brooklyn"')
soql_health_steward_qu = soql_health_steward.query('boroname == "Queens"')
soql_health_steward_mn = soql_health_steward.query('boroname == "Manhattan"')
soql_health_steward_si = soql_health_steward.query('boroname == "Staten Island"')

# URL

In [97]:
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$select=distinct spc_common&$order=spc_common').replace(' ','%20')
soql_trees = pd.read_json(soql_url).dropna()

soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$select=distinct%20boroname&$order=boroname').replace(' ', '%20')
soql_boros = pd.read_json(soql_url).dropna()


# App Layout

In [77]:
app = JupyterDash('HW4', external_stylesheets=external_stylesheets)

In [98]:
app.layout = html.Div([
    html.Label('Tree Species'),
    dcc.Dropdown(
        id='spc',
        options=[
            {'label': s, 'value': s} for s in soql_trees['spc_common']
        ]
    ),
    html.Label('Borough'),
    dcc.Dropdown(
        id='bor',
        options=[
            {'label': s, 'value': s} for s in soql_boros['boroname']
        ]
    ),
    html.Div(id='my-div'),
    dcc.Checklist(
        id='stw',
        options=[{'label': 'Steward', 'value': 'Y'}]
    ),
    dcc.Graph(id='graph-1')
])


# app callback


In [109]:
import plotly.express as px
import plotly.graph_objects as go
from dash import callback as ctx

In [105]:
@app.callback(
    Output(component_id='my-div', component_property='children'),
    [Input(component_id='spc', component_property='value'),
     Input(component_id='bor', component_property='value')],
     prevent_initial_call=True
)
def update_output_div(tree_spc, boroname):
    if (tree_spc and boroname):
        return 'You\'ve selected "{}" in {}'.format(tree_spc, boroname)
    else:
        return 'Please complete selection'

@app.callback(
    Output(component_id='graph-1', component_property='figure'),
    [Input(component_id='spc', component_property='value'),
     Input(component_id='bor', component_property='value'),
     Input(component_id='stw', component_property='value')],
     prevent_initial_call=True
)
def update_output_graph_1(tree_spc, boroname, steward):
    triggered_id = ctx.triggered_id
    print(triggered_id)
    if triggered_id == 'tree_spc' and 'boroname' and 'steward':
        return get_figure_by_steward_graph_1(tree_spc, boroname)
    elif triggered_id == 'tree_spc' and 'boroname':
        soql_url = ("https://data.cityofnewyork.us/resource/uvpi-gqnh.json?" +\
            "$select=health,count(spc_common)&" +\
            "$where=health is not null and spc_common='{0}' and boroname='{1}'&" +\
            "$group=health").format(tree_spc, boroname).replace(' ', '%20')
        df = pd.read_json(soql_url)
        sum_spc = sum(df['count_spc_common'])
        return {
            'data': [
                {'x': df['health'], 'y': df['count_spc_common'], 'type': 'bar', 'name': 'None'},
            ],
            'layout': {
                'title': 'Proportions by Health of %d "%s" Trees in %s' % (sum_spc, tree_spc, boroname)
            }
        }
    else:
        raise PreventUpdate
        
def get_figure_by_steward_graph_1(tree_spc, boroname):
    soql_url = ("https://data.cityofnewyork.us/resource/uvpi-gqnh.json?" +\
            "$select=steward,health,count(steward)&" +\
            "$where=health is not null and steward is not null and spc_common='{0}' and boroname='{1}'&" +\
            "$group=steward,health").format(tree_spc, boroname).replace(' ', '%20')
    df = pd.read_json(soql_url)
    sum_spc = sum(df['count_steward'])
    lst = list()
    for s in df['steward'].unique():
        lst.append({'x': df[df['steward'] == s]['health'], 'y': df[df['steward'] == s]['count_steward'], 'type': 'bar', 'name': 'Steward: %s' % s})
    return {
            'data': lst,
            'layout': {
                'barmode': 'stack',
                'title': 'Proportions by Health of %d "%s" Trees in %s' % (sum_spc, tree_spc, boroname)
            }
        }

In [107]:

    app.run_server(mode="external")

Dash app running on:


<IPython.core.display.Javascript object>

In [114]:
fig = px.bar(soql_health, x="spc_common", y="count",color="health")
fig.show()

In [119]:
fig = px.bar(soql_health_steward, x="spc_common", y="count",color="steward")
fig.show()